# WSW Spike Charts

Iterate through folders, create charts for sessions affected by spikes.

Copyright 2022 Michael George (AKA Logiqx).

This file is part of [GPS Wizard](https://github.com/Logiqx/gps-wizard) and is distributed under the terms of the GNU General Public License.

GPS Wizard is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

GPS Wizard is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with GPS Wizard. If not, see <https://www.gnu.org/licenses/>.

## Notes

Sessions have been pre-identified and placed into a seperate folder.

In [450]:
import os
import sys
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as tck

import traceback

corePath = os.path.join('..', 'core')
if corePath not in sys.path:
    sys.path.extend([corePath])

from file_reader import getFileReader

## Main Function

In [451]:
def processFiles():
    '''Iterate through session archive creating charts for each GPS file'''

    rootDir = os.path.join(projdir, '..', 'gps-spikes', 'gpslogs', 'motion', 'wsw')

    errors = {}
    
    for root, subDirs, files in os.walk(rootDir):
        for file in files:
            ext = os.path.splitext(file)[1].lower()
            
            if ext in ['.oao'] and 'STA867SCO_867_20231013_083816' in file:
                filePath = os.path.join(root, file)
                print(filePath)
                reader = getFileReader(filePath)
                try:
                    # Some legacy ESP-GPS files contain bad checksums
                    if ext == '.ubx':
                        reader.load(ignoreChecksums=True)
                    else:
                        reader.load()
                    
                    # Process all tracks within the file
                    for track in reader.tracks:
                        createCharts(track)
                        print('.', end='')
                        
                except Exception:
                    errors[filePath.replace(projdir + '/', '')] = traceback.format_exc()
                    print('E', end='')

    if len(errors) > 0:
        print(os.linesep * 2 + 'Errors:')
        for error in errors:
            print(error)
            print(errors[error])

In [524]:
# Factor to convert m/s to knots
MS_TO_KNOTS = 3600 / 1852

# Maximum timestamp difference in milliseconds
MAX_TIMESTAMP_DIFF = 200

# Filter values
HDOP_THRESHOLD = 20
EHPE_THRESHOLD = 25
EHVE_THRESHOLD = 1.2 * MS_TO_KNOTS

charts = [
    {
        'title': 'Speed over Ground (SOG)',
        'label': 'Speed (knots)',
        'field': 'sog',
        'scale': MS_TO_KNOTS
    },
    {
        'title': 'Number of Satellites',
        'label': 'Satellites',
        'field': 'sat'
    },
    {
        'title': 'Horizontal Dilution of Precision (HDOP)',
        'label': 'HDOP',
        'field': 'hdop'
    },
    {
        'title': 'Positional Dilution of Precision (PDOP)',
        'label': 'pDOP',
        'field': 'pdop'
    },
    {
        'title': 'Horizontal Accuracy (hAcc)',
        'label': 'hAcc (m)',
        'field': 'ehpe'
    },
    {
        'title': 'Speed Accuracy (sAcc)',
        'label': 'sAcc (knots)',
        'field': 'ehve',
        'scale': MS_TO_KNOTS
    }
]

def createSubPlot(fig, ax, title, ylabel, xPoints, yPoints):
    '''Create charts for a GPS track'''
    
    # Titles and axis labels
    ax.set_title(title, y=1.01, fontsize=12)
    ax.set_ylabel(ylabel, fontsize=11)   
    ax.get_xaxis().set_major_formatter(tck.FuncFormatter(lambda x, p: datetime.fromtimestamp(x).strftime('%H:%M')))
    
    # Trick to ensure missing data points are plotted in grey
    ax.plot(xPoints, yPoints, color='silver', linestyle='dotted')

    # Plot the actual data points in blue
    segmentIdx = -1
    prevTs = 0
    for i in range(len(xPoints)):
        currTs = xPoints[i]
        if i > 0:
            if ((currTs - prevTs) * 1000).astype(int) > MAX_TIMESTAMP_DIFF:
                if segmentIdx >= 0:
                    ax.plot(xPoints[segmentIdx:i], yPoints[segmentIdx:i], color='#1f77b4')
                    segmentIdx = -1
            else:
                if segmentIdx < 0:
                    segmentIdx = i
        prevTs = currTs

    if segmentIdx >= 0:
        ax.plot(xPoints[segmentIdx:i], yPoints[segmentIdx:i], color='#1f77b4')

    # Determine number of tick intervals
    duration = xPoints.max() - xPoints.min()
    tickInterval = 900   
    while duration // tickInterval > 12:
        tickInterval *= 2
        
    # Determine first and last tick
    firstTick = xPoints.min() // tickInterval * tickInterval
    if firstTick < xPoints.min():
        firstTick += tickInterval
    lastTick = xPoints.max() // tickInterval * tickInterval + 1
    ax.set_xticks(np.arange(firstTick, lastTick, tickInterval))
    
    # Add grid lines
    ax.grid(which='major', axis='y')
                      
    # Ensure y-axis starts at zero
    ax.set_ylim(ymin=0)


def createChart(filePath, data, description, pngName):
    '''Create charts for a GPS track'''

    numAxs = 0
    for chart in charts:
        if chart['field'] in data:
            numAxs += 1
    
    fig, axs = plt.subplots(numAxs, figsize=(16, 24), dpi=100)
    plt.subplots_adjust(hspace=0.3)
    
    filename = os.path.basename(filePath)
    plt.suptitle('{}'.format(filename), y=0.925, fontsize=18, fontweight='bold')

    if description:
        textStr = 'Filtered by {}'.format(description)
    else:
        textStr = 'Unfiltered'

    fig.text(0.5, 0.905, textStr, fontsize=14, fontweight='bold', horizontalalignment='center')

    # Determine typical timestamp difference in ms - e.g. 200 for Motions
    tsDiffs = ((data['ts'][1:] - data['ts'][:-1]) * 1000).astype(int)
    uniqueValues, counts = np.unique(tsDiffs, return_counts=True)
    typicalTsDiff = uniqueValues[np.argmax(counts)]

    # X points is simply the timestamps
    xPoints = data['ts']
    
    # Generate each of the charts
    axsId = 0
    for i in range(len(charts)):
        chart = charts[i]

        if chart['field'] in data:
            createSubPlot(fig, axs[axsId], chart['title'], chart['label'], xPoints, data[chart['field']])
            axsId += 1
        
    # Ensure directory for PNG exists
    dirName = os.path.join(os.path.dirname(filePath), 'img', os.path.basename(filePath))
    if not os.path.exists(dirName):
        os.makedirs(dirName)
        
    # Save PNG
    fileName = os.path.join(dirName, pngName + '.png')
    print(f"Saving {fileName}...")
    fig.savefig(fileName, bbox_inches='tight', facecolor='w')

    # Ensure the charts do not appear in Jupyter
    plt.close(fig)


def getFilteredData(unfilteredData, points):
    '''Get filtered data'''
    
    filteredData = {}

    filteredData['ts'] = unfilteredData['ts'][points]

    for chart in charts:
        if chart['field'] in unfilteredData:
            filteredData[chart['field']] = unfilteredData[chart['field']][points]
        
    return filteredData

    
def processFile(filePath, track):
    '''Process a GPS file'''

    unfilteredData = {}
    filteredData = {}
    
    # Take a copy of the data and convert m/s to knots, where applicable
    unfilteredData['ts'] = track.data['ts']

    for chart in charts:
        if chart['field'] in track.data:
            unfilteredData[chart['field']] = track.data[chart['field']]

            if 'scale' in chart:
                unfilteredData[chart['field']] *= chart['scale']

    # Unfiltered
    createChart(filePath, unfilteredData, None, 'raw')
    
    # Ignore "no fix"
    points = np.argwhere(track.data['fix'] > 0)
    filteredData = getFilteredData(unfilteredData, points)
    filterDesc = 'fix > 0'
    createChart(filePath, filteredData, filterDesc, 'on-fix')
    
    # Apply EHPE filter
    points = np.argwhere((track.data['fix'] > 0) &
                            (unfilteredData['ehpe'] <= EHPE_THRESHOLD))
    filteredData = getFilteredData(unfilteredData, points)
    filterDesc = 'fix > 0 and hAcc <= {:.01f} m'.format(EHPE_THRESHOLD)
    createChart(filePath, filteredData, filterDesc, 'hacc')

    # Apply HDOP + EHPE filter
    #points = np.argwhere((track.data['fix'] > 0) &
    #                        (unfilteredData['ehpe'] <= EHPE_THRESHOLD)  &
    #                        (unfilteredData['hdop'] <= HDOP_THRESHOLD))
    #filteredData = getFilteredData(unfilteredData, points)
    #filterDesc = 'fix > 0 and HDOP <= 20 and hAcc <= {:.01f} m'.format(EHPE_THRESHOLD)
    #createChart(filePath, filteredData, filterDesc, 'fix-hdop-hacc')


if __name__ == '__main__':
    projdir = os.path.realpath(os.path.join(sys.path[0], "..", ".."))

    # WSW
    #filePath = '/home/jovyan/work/gps-spikes/gpslogs/motion/wsw/ashore/STA867SCO_867_20231013_083816.oao'
    #filePath = '/home/jovyan/work/gps-spikes/gpslogs/motion/wsw/ashore/KNI841FIN_841_20231007_103000.oao'
    #filePath = '/home/jovyan/work/gps-spikes/gpslogs/motion/wsw/ashore/TUR808AND_808_20231013_084118.oao'
    #filePath = '/home/jovyan/work/gps-spikes/gpslogs/motion/wsw/ashore/CRO632PET_632_20231013_092740.oao'
    #filePath = '/home/jovyan/work/gps-spikes/gpslogs/motion/wsw/ashore/DUN813ROB_813_20231013_094502.oao'

    # Wing dead reckoning
    #filePath = '/home/jovyan/work/gps-wizard/sessions/brog-wing-spike/2940_2022-10-26-1321.oao'
    
    # Motion spike
    #filePath = '/home/jovyan/work/gps-wizard/sessions/motion-spike/2936_2023-06-14-1155.oao'
    
    # Manfred 20 Hz
    #filePath = '/home/jovyan/work/gps-wizard/sessions/manfred-20hz/in/093154.UBx'
    #ignoreChecksums = True
    
    reader = getFileReader(filePath)
    reader.load(ignoreChecksums=ignoreChecksums)
    track = reader.tracks[0]
    
    processFile(filePath, track)

Saving /home/jovyan/work/gps-spikes/gpslogs/motion/wsw/ashore/img/DUN813ROB_813_20231013_094502.oao/raw.png...
Saving /home/jovyan/work/gps-spikes/gpslogs/motion/wsw/ashore/img/DUN813ROB_813_20231013_094502.oao/on-fix.png...
Saving /home/jovyan/work/gps-spikes/gpslogs/motion/wsw/ashore/img/DUN813ROB_813_20231013_094502.oao/hacc.png...


In [453]:
if __name__ == '__main__':
    projdir = os.path.realpath(os.path.join(sys.path[0], "..", ".."))

    #processFiles()
    
    print(os.linesep + 'All done!')


All done!
